# Pancreatic Cancer Prediction

### Background and Goal

Pancreatic cancer (PC) is the fourth leading cause of cancer-related death in both men and women despite its low incidence rate. PC is known to be as a silent killer as it commonly get diagnosed in late stages.

The 5-years survival rate for all stages of PC increased to 10% by the end of 2019,while it was 8.5% in 2017. The 5-years survival rates for patients with early-stage diagnosis can be as high as 20%. However, only a small portion of patients have surgically resectable disease at the time of diagnosis). Furthermore, identification of individuals at high risk for PC or with early-stage disease is difficult due to the lack of a reliable screening tools, the absence of sensitive and specific biomarkers, and the low prevalence.[2]

Recently, numerous studies have been focused on early detection of PC through the identification and validation of promising biomarkers, imigaing utilities,


### Task

We want to demonstrate the feasibility of different deep learning architectures mainly Recurrent Neural Networks(RNN) for pancreatic cancer prediction and compare against other baseline models like Logistic Regression (LR) and Random Forrest (RF). 

### Dataset

We will use the Cerner HealthFacts® database and we will extract patients >45 years old with at least two inpatient encounters and has reported pancreatic cancer as our cases, and will extract a similar number of control patients who are also older than 45 years old, but they didn't get diagnosed with any cancer disease and never died or transfered to hospice as appears in the dataset

### Metrics
We will use the ROC AUC for the models’ performance comparison.


In [1]:
'''
### Notes from the class

--- use argparse create automtic help versus sys.arg
'''

'\n### Notes from the class\n\n--- use argparse create automtic help versus sys.arg\n'

#### Required Tools and Packages
---------------------------------
We mostly use scikit-learn for baseline machine learning, plotly and matpoltlib for visualization within ipython notebooks, but we also use Tableau aside of it.

For deep learning we rely on Pytorch framework (v1), GPU enabling accelerate the computational performance,and for the full set model trainings we might need to utilize the Pytorch DataParallel functionality for parallel GPU computing. 

In [2]:
### Tools and Packages
##Basics
import pandas as pd
import numpy as np
import sys, random
import math
try:
    import cPickle as pickle
except:
    import pickle
import string
import re
import os
import time
from tqdm import tqdm

## ML and Stats 
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import sklearn.metrics as m
import sklearn.linear_model  as lm
import lifelines#.estimation import KaplanMeierFitter
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import export_graphviz
import statsmodels.formula.api as sm
import patsy
from scipy import stats

## Visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.graph_objs import *
from IPython.display import HTML

## DL Framework
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim



###GPU enabling and device allocation
use_cuda = torch.cuda.is_available()
torch.cuda.set_device(7)



# Baseline Analysis

#### Model Definitions

In [3]:
##### Data conversion to onehot matrices for Logestic Regression and may be Random Forest Basic test
def run_LR_model(train_sl,test_sl,input_size_1):
    pts_tr=[]
    labels_tr=[]
    features_tr=[]
    for pt in train_sl:
        pts_tr.append(pt[0])
        labels_tr.append(pt[1])
        x=[]
        for v in pt[-1]:
            x.extend(v[-1])
        features_tr.append(x)

    pts_t=[]
    labels_t=[]
    features_t=[]
    for pt in test_sl:
        pts_t.append(pt[0])
        labels_t.append(pt[1])
        x=[]
        for v in pt[-1]:
            x.extend(v[-1])
        features_t.append(x)


    mlb = MultiLabelBinarizer(classes=range(input_size_1[0])[1:])
    nfeatures_tr = mlb.fit_transform(features_tr)
    nfeatures_t= mlb.fit_transform(features_t)

    EHR_LR= lm.LogisticRegression()
    EHR_LR.fit(nfeatures_tr, labels_tr)
    #labels,scores=evaluate(EHR_LR,nfeatures_t, labels_t)
    #predictions = EHR_LR.predict(test_features)
    pred_prob=EHR_LR.predict_proba(nfeatures_t)
    auc_p=roc_auc_score(labels_t,pred_prob[:,1])
    return auc_p,pts_t,labels_t,pred_prob[:,1]


In [4]:
### DL RNN Model training 
import models as model 
from EHRDataloader import EHRdataloader
from EHRDataloader import EHRdataFromLoadedPickles as EHRDataset
import utils as ut 
from EHREmb import EHREmbeddings

### for tracking computational timing
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def run_dl_model(ehr_model,train_sl,valid_sl,test_sl,bmodel_pth,bmodel_st,wmodel,packpadmode):
    ## Data Loading
    print (' creating the list of training minibatches')
    train = EHRDataset(train_sl,sort= True, model='RNN')
    train_mbs = list(tqdm(EHRdataloader(train, batch_size = 128, packPadMode = packpadmode)))
    print (' creating the list of valid minibatches')
    valid = EHRDataset(valid_sl,sort= True, model='RNN')
    valid_mbs = list(tqdm(EHRdataloader(valid, batch_size = 128, packPadMode = packpadmode)))
    print (' creating the list of test minibatches')
    test = EHRDataset(test_sl,sort= True, model='RNN')
    test_mbs = list(tqdm(EHRdataloader(test, batch_size = 128, packPadMode = packpadmode)))


    ##Hyperparameters -- Fixed for testing purpose
    epochs = 100
    l2 = 0.0001
    lr = 0.01
    eps = 1e-4
    w_model= wmodel
    optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps)   

    ##Training epochs
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
    train_auc_allep =[]
    valid_auc_allep =[]
    test_auc_allep=[]  
    for ep in range(epochs):
        start = time.time()
        current_loss, train_loss = ut.trainbatches(train_mbs, model= ehr_model, optimizer = optimizer)
        avg_loss = np.mean(train_loss)
        train_time = timeSince(start)
        eval_start = time.time()
        Train_auc, y_real, y_hat  = ut.calculate_auc(ehr_model, train_mbs, which_model = w_model)
        valid_auc, y_real, y_hat  = ut.calculate_auc(ehr_model, valid_mbs, which_model = w_model)
        TestAuc, y_real, y_hat = ut.calculate_auc(ehr_model, test_mbs, which_model = w_model)
        eval_time = timeSince(eval_start)
        print ("Epoch: " ,str(ep) ," Train_auc :" , str(Train_auc) , " , Valid_auc : " ,str(valid_auc) , " ,& Test_auc : " , str(TestAuc) ," Avg Loss: " ,str(avg_loss), ' , Train Time :' , str(train_time) ,' ,Eval Time :' ,str(eval_time))
        train_auc_allep.append(Train_auc)
        valid_auc_allep.append(valid_auc)
        test_auc_allep.append(TestAuc)
        if valid_auc > bestValidAuc: 
            bestTrainAuc=Train_auc
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            bestTestAuc = TestAuc
            ###uncomment the below lines to save the best model parameters
            best_model = ehr_model
            torch.save(best_model, bmodel_pth)
            torch.save(best_model.state_dict(), bmodel_st)
        if ep - bestValidEpoch >10: break
    print( 'bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))
    return bestTrainAuc,bestValidAuc,bestTestAuc,bestValidEpoch

### Run Models

In [5]:
train_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.train', 'rb'), encoding='bytes')
test_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.test', 'rb'), encoding='bytes')
valid_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.valid', 'rb'), encoding='bytes')
types_d=pickle.load(open('../data/pdata/lr_pc_cid.types', 'rb'), encoding='bytes')

print (len(train_sl))

20584


In [5]:
results=[]
dlcalc_RNN=[]
dlcalc_LR=[]
for term in ['csr','cid','icd9','icd10','ccs','uml','phe']:
    if term=='cid':
        train_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_cid.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_cid.types', 'rb'), encoding='bytes')
    elif term=='icd9':
        train_sl= pickle.load(open('../data/pdata/lr_pc_icd9_n.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_icd9_n.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_icd9_n.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_icd9_n.types.valid', 'rb'), encoding='bytes')
    elif term=='icd10':
        train_sl= pickle.load(open('../data/pdata/lr_pc_icd10_n.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_icd10_n.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_icd10_n.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_icd10_n.types.valid', 'rb'), encoding='bytes')
    elif term=='ccs':
        train_sl= pickle.load(open('../data/pdata/lr_pc_ccs_n.combinedtrain', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_ccs_n.combinedtest', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_ccs_n.combinedvalid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_ccs_n.types.valid', 'rb'), encoding='bytes')
    elif term=='uml':
        train_sl= pickle.load(open('../data/pdata/lr_pc_umls_n.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_umls_n.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_umls_n.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_umls_n.types.valid', 'rb'), encoding='bytes')
    elif term=='phe':
        train_sl= pickle.load(open('../data/pdata/lr_pc_phe_v2.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_phe_v2.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_phe_v2.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_phe_v2.types.valid', 'rb'), encoding='bytes')
    elif term=='csr':
        train_sl= pickle.load(open('../data/pdata/lr_pc_ccsr.combined.train', 'rb'), encoding='bytes')
        test_sl= pickle.load(open('../data/pdata/lr_pc_ccsr.combined.test', 'rb'), encoding='bytes')
        valid_sl= pickle.load(open('../data/pdata/lr_pc_ccsr.combined.valid', 'rb'), encoding='bytes')
        types_d=pickle.load(open('../data/pdata/lr_pc_ccsr.types.train', 'rb'), encoding='bytes')

    types_d_rev = dict(zip(types_d.values(),types_d.keys()))
    input_size_1=[len(types_d_rev)+1]
    for run in range(10):
        LR_auc,pts_tsk,labelst,predprob=run_LR_model(train_sl,test_sl,input_size_1)
        dlcalc_LR.append({'Term':term,'Run':run,'pt_sk':pts_tsk,'label':labelst,'lR_prob':predprob})
        ehr_model = model.EHR_RNN(input_size_1, embed_dim=64, hidden_size=64, n_layers=1, dropout_r=0., cell_type='GRU', bii=True , time=True)
        if use_cuda: ehr_model = ehr_model.cuda()    
        bestTrainAuc,bestValidAuc,bestTestAuc,bestValidEpoch=run_dl_model(ehr_model,train_sl,valid_sl,test_sl,term+str(run)+'Panc_BiGRU.pth',term+str(run)+'panc_BiGRU.st',wmodel='RNN',packpadmode=True)
        results.append([term,run,LR_auc,bestTrainAuc,bestValidAuc,bestTestAuc,bestValidEpoch])
        
        best_ehr_model=torch.load(term+str(run)+'Panc_BiGRU.pth')
        best_ehr_model.load_state_dict(torch.load(term+str(run)+'panc_BiGRU.st'))

        for pt in test_sl: #### I do that loop to get the result per Pt_sk (current dataloader don't dump the pt_sk)
            test_dlg = EHRDataset([pt],sort= True, model='RNN')
            sample, label_tensor, seq_l, mtd = list(EHRdataloader(test_dlg, batch_size =1, packPadMode = True))[0]
            output = best_ehr_model(sample, seq_l, mtd)
            y_hat=output.cpu().data.view(-1).numpy()  
            y_real=label_tensor.cpu().data.view(-1).numpy()
            dlcalc_RNN.append([term,run,pt[0],y_real[0],y_hat[0]]) 




  0%|          | 0/161 [00:00<?, ?it/s]

 creating the list of training minibatches


  0%|          | 0/23 [00:00<?, ?it/s]

 creating the list of valid minibatches


  0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/161 [00:00<?, ?it/s]

 creating the list of training minibatches


  0%|          | 0/23 [00:00<?, ?it/s]

 creating the list of valid minibatches


 43%|████▎     | 10/23 [00:01<00:01,  7.21it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Create Summary descripton for TEST AUC 

In [6]:
df=pd.DataFrame(results)
df.columns=['Term','Run','LR_auc','bestTrainAuc','bestValidAuc','bestTestAuc','bestValidEpoch']
df.to_csv('PC_Term_results_r4.csv')
LR_sum=df[['Term','LR_auc']].groupby(['Term']).describe()
RNN_sum=df[['Term','bestTestAuc']].groupby(['Term']).describe()
LR_sum.to_csv('PC_Term_results_r4_LR_sum.csv')
RNN_sum.to_csv('PC_Term_resultsh_r4_RNN_sum.csv')

In [7]:
LR_sum

LR_auc                                                                  \
       count      mean           std       min       25%       50%       75%   
Term                                                                           
ccs     10.0  0.772305  3.834110e-08  0.772305  0.772305  0.772305  0.772305   
cid     10.0  0.802977  2.300466e-07  0.802977  0.802978  0.802978  0.802978   
csr     10.0  0.779204  1.035210e-06  0.779204  0.779204  0.779204  0.779204   
icd10   10.0  0.789484  5.751166e-07  0.789482  0.789484  0.789484  0.789484   
icd9    10.0  0.791476  1.170278e-16  0.791476  0.791476  0.791476  0.791476   
phe     10.0  0.788668  3.453161e-07  0.788668  0.788668  0.788668  0.788668   
uml     10.0  0.805255  2.300466e-07  0.805255  0.805255  0.805255  0.805255   

                 
            max  
Term             
ccs    0.772305  
cid    0.802978  
csr    0.779207  
icd10  0.789484  
icd9   0.791476  
phe    0.788669  
uml    0.805255

In [8]:
RNN_sum

bestTestAuc                                                              \
            count      mean       std       min       25%       50%       75%   
Term                                                                            
ccs          10.0  0.790253  0.003576  0.784379  0.788432  0.789780  0.792995   
cid          10.0  0.814258  0.003709  0.806083  0.813619  0.814369  0.816183   
csr          10.0  0.796316  0.003399  0.791040  0.793675  0.797297  0.798358   
icd10        10.0  0.792745  0.004447  0.785249  0.790707  0.791960  0.795565   
icd9         10.0  0.807835  0.003176  0.800666  0.807104  0.808617  0.809503   
phe          10.0  0.811946  0.004960  0.804033  0.808158  0.812742  0.813923   
uml          10.0  0.822450  0.002944  0.817076  0.820364  0.823746  0.824035   

                 
            max  
Term             
ccs    0.795501  
cid    0.820203  
csr    0.800822  
icd10  0.800157  
icd9   0.811513  
phe    0.820555  
uml    0.826585

#### Calculate Delong Pvalue

In [9]:
##### Results Preparation
###RNN results
RNN_results_DL=pd.DataFrame(dlcalc_RNN)
RNN_results_DL.columns=['Term','Run','Pt_sk','label','RNN_prob']
RNN_results_pv=RNN_results_DL[~(RNN_results_DL['Pt_sk'].isin([126988858,196334846,206308992]))].pivot_table(index=['Pt_sk','label'], columns='Term', values='RNN_prob').reset_index()
### LR results
LR_results_DL = pd.concat([pd.DataFrame(di) for di in dlcalc_LR], axis=0)
LR_results_pv=LR_results_DL[~(LR_results_DL['pt_sk'].isin([126988858,196334846,206308992]))].pivot_table(index=['pt_sk','label'], columns='Term', values='lR_prob').reset_index()

In [10]:
#RNN_results_pv.isnull().sum()### 1pt in icd 9 [126988858] and 2 in Phe [196334846,206308992](exclude those 3)
#RNN_results_pv[RNN_results_pv['phe'].isnull()] 

In [11]:
import Delong_Calc as dlgc
dl_results=[]
for term1 in ['csr','cid','icd9','icd10','ccs','uml','phe']:
    for term2 in ['csr','cid','icd9','icd10','ccs','uml','phe']:
        pval_LR=dlgc.delong_roc_test(LR_results_pv['label'].to_numpy(),LR_results_pv[term1].to_numpy(),LR_results_pv[term2].to_numpy())
        pval_RNN=dlgc.delong_roc_test(RNN_results_pv['label'].to_numpy(),RNN_results_pv[term1].to_numpy(),RNN_results_pv[term2].to_numpy())
        #print (term1, 'vs', term2,'LR log p-value=', pval_LR[0][0] ,'RNN log p-value=', pval_RNN[0][0])
        dl_results.append([term1, term2, pval_LR[0][0], pval_RNN[0][0]])

In [12]:
'''cid vs cid LR log p-value= nan RNN log p-value= nan
cid vs icd9 LR log p-value= -4.223370262120469 RNN log p-value= -4.043063612205202
cid vs icd10 LR log p-value= -4.834255676332444 RNN log p-value= -3.809331316475661
cid vs ccs LR log p-value= -6.618930497528267 RNN log p-value= -11.749515389592565
cid vs uml LR log p-value= -0.5411129252816755 RNN log p-value= -0.2880030779515665
icd9 vs cid LR log p-value= -4.223370262120469 RNN log p-value= -4.043063612205202
icd9 vs icd9 LR log p-value= nan RNN log p-value= nan
icd9 vs icd10 LR log p-value= -0.2957389927105521 RNN log p-value= -0.08267607934863302
icd9 vs ccs LR log p-value= -3.089835031814745 RNN log p-value= -4.962188184271849
icd9 vs uml LR log p-value= -4.801938956218903 RNN log p-value= -2.783956724545916
icd10 vs cid LR log p-value= -4.834255676332444 RNN log p-value= -3.809331316475661
icd10 vs icd9 LR log p-value= -0.2957389927105521 RNN log p-value= -0.08267607934863302
icd10 vs icd10 LR log p-value= nan RNN log p-value= nan
icd10 vs ccs LR log p-value= -2.5344256288301414 RNN log p-value= -4.971236424636719
icd10 vs uml LR log p-value= -5.548689280926472 RNN log p-value= -2.472776122430077
ccs vs cid LR log p-value= -6.618930497528267 RNN log p-value= -11.749515389592565
ccs vs icd9 LR log p-value= -3.089835031814745 RNN log p-value= -4.962188184271849
ccs vs icd10 LR log p-value= -2.5344256288301414 RNN log p-value= -4.971236424636719
ccs vs ccs LR log p-value= nan RNN log p-value= nan
ccs vs uml LR log p-value= -7.831945346986503 RNN log p-value= -11.271779865126106
uml vs cid LR log p-value= -0.5411129252816755 RNN log p-value= -0.2880030779515665
uml vs icd9 LR log p-value= -4.801938956218903 RNN log p-value= -2.783956724545916
uml vs icd10 LR log p-value= -5.548689280926472 RNN log p-value= -2.472776122430077
uml vs ccs LR log p-value= -7.831945346986503 RNN log p-value= -11.271779865126106
uml vs uml LR log p-value= nan RNN log p-value= nan
'''

'cid vs cid LR log p-value= nan RNN log p-value= nan\ncid vs icd9 LR log p-value= -4.223370262120469 RNN log p-value= -4.043063612205202\ncid vs icd10 LR log p-value= -4.834255676332444 RNN log p-value= -3.809331316475661\ncid vs ccs LR log p-value= -6.618930497528267 RNN log p-value= -11.749515389592565\ncid vs uml LR log p-value= -0.5411129252816755 RNN log p-value= -0.2880030779515665\nicd9 vs cid LR log p-value= -4.223370262120469 RNN log p-value= -4.043063612205202\nicd9 vs icd9 LR log p-value= nan RNN log p-value= nan\nicd9 vs icd10 LR log p-value= -0.2957389927105521 RNN log p-value= -0.08267607934863302\nicd9 vs ccs LR log p-value= -3.089835031814745 RNN log p-value= -4.962188184271849\nicd9 vs uml LR log p-value= -4.801938956218903 RNN log p-value= -2.783956724545916\nicd10 vs cid LR log p-value= -4.834255676332444 RNN log p-value= -3.809331316475661\nicd10 vs icd9 LR log p-value= -0.2957389927105521 RNN log p-value= -0.08267607934863302\nicd10 vs icd10 LR log p-value= nan RNN

In [13]:
DL_results_df=pd.DataFrame(dl_results)
DL_results_df.columns=['Term1','Term2','LR_log_pv','RNN_log_pv']

In [14]:
LR_results_DL.to_csv('PC_LR_testpts_results_r4.csv')
RNN_results_DL.to_csv('PC_RNN_testpts_results_r4.csv')
DL_results_df.to_csv('PC_Delonglogpvalues_results_r4.csv')

In [15]:
DL_results_df

,Term1,Term2,LR_log_pv,RNN_log_pv
0,csr,csr,NaN,NaN
1,csr,cid,-4.599914,-6.666705
2,csr,icd9,-1.608384,-3.949753
3,csr,icd10,-1.218575,-0.724053
4,csr,ccs,-1.053725,-0.511812
5,csr,uml,-5.606620,-9.580295
6,csr,phe,-1.220955,-6.080431
7,cid,csr,-4.599914,-6.666705
8,cid,cid,NaN,NaN
9,cid,icd9,-4.219207,-1.831780


#### Calculate Unpaired T-Test

In [16]:
from scipy import stats



In [17]:
tt_results=[]
for term1 in RNN_sum.index:
    mean1=RNN_sum.loc[term1,('bestTestAuc',  'mean')]
    std1= RNN_sum.loc[term1,('bestTestAuc',  'std')]
    nobs1=RNN_sum.loc[term1,('bestTestAuc',  'count')]
    for term2 in RNN_sum.index:
        if term1==term2: continue
        else:
            #mean2=RNN_sum.loc[term2,('bestTestAuc',  'mean')]
            #std2= RNN_sum.loc[term2,('bestTestAuc',  'std')]
            #nobs2=RNN_sum.loc[term2,('bestTestAuc',  'count')]
            sts1,pvalue1=stats.ttest_ind_from_stats(mean1, std1, nobs1 ,RNN_sum.loc[term2,('bestTestAuc',  'mean')],RNN_sum.loc[term2,('bestTestAuc',  'std')], RNN_sum.loc[term2,('bestTestAuc',  'count')], equal_var=True)
            sts2,pvalue2=stats.ttest_ind_from_stats(mean1, std1, nobs1 ,RNN_sum.loc[term2,('bestTestAuc',  'mean')],RNN_sum.loc[term2,('bestTestAuc',  'std')], RNN_sum.loc[term2,('bestTestAuc',  'count')], equal_var=False)
            tt_results.append([term1 , term2, sts1,pvalue1,sts2,pvalue2])                                                                                                                   

In [18]:
tt_df=pd.DataFrame(tt_results)
tt_df.columns=['term1' , 'term2', 'sts1','pvalue1','sts2','pvalue2']
tt_df.to_csv('PC_RNN_TT_pvalues_results_r4.csv')

In [19]:
RNN_sum

bestTestAuc                                                              \
            count      mean       std       min       25%       50%       75%   
Term                                                                            
ccs          10.0  0.790253  0.003576  0.784379  0.788432  0.789780  0.792995   
cid          10.0  0.814258  0.003709  0.806083  0.813619  0.814369  0.816183   
csr          10.0  0.796316  0.003399  0.791040  0.793675  0.797297  0.798358   
icd10        10.0  0.792745  0.004447  0.785249  0.790707  0.791960  0.795565   
icd9         10.0  0.807835  0.003176  0.800666  0.807104  0.808617  0.809503   
phe          10.0  0.811946  0.004960  0.804033  0.808158  0.812742  0.813923   
uml          10.0  0.822450  0.002944  0.817076  0.820364  0.823746  0.824035   

                 
            max  
Term             
ccs    0.795501  
cid    0.820203  
csr    0.800822  
icd10  0.800157  
icd9   0.811513  
phe    0.820555  
uml    0.826585